In [1]:
# Colab 환경 설정
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece
#!pip install torch
!pip install transformers
!pip install numpy==1.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661762 sha256=799b225cd25f93e782b34ec164076019397c2bfb93dc070c77d37adcd998e030
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-91yjv60i/kobert-tokenizer_5d01e410df50495ba00a4fe97f26377a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-91yjv60i/kobert-tokenizer_5d01e410df50495ba00a4fe97f26377a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=dab6479bafcca25ea498b2629a3164dfec09839852cdd9a4a397c6d7e131f745
  Stored in directory: /tmp/pip-ephem-wheel-cache-y65pf4u0/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
#KoBERT
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
#transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split


In [4]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [5]:
from transformers import BertForSequenceClassification
model =  BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=3, # Binary classification (Positive and Negative)
    output_attentions=False,
    output_hidden_states=False,
)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [10]:
from sklearn.model_selection import train_test_split

train = pd.read_excel('/content/drive/MyDrive/감성대화말뭉치(최종데이터)_Training.xlsx')
# valid = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Validation.xlsx')

# train['문장'] = train['사람문장1']+ train['시스템문장1']+train['사람문장2']+train['시스템문장2']
train['문장'] = train['사람문장1']+train['사람문장2']
train = train.loc[:,['감정_대분류', '문장']]
train.loc[train.감정_대분류 == '기쁨', '감정_대분류'] = 1
train.loc[train.감정_대분류 != 1, '감정_대분류'] = -1

train['sentiment'] = train['감정_대분류']
train['content'] = train['문장']

train.drop(columns=['감정_대분류', '문장'], inplace=True)
train['sentiment'] = train['sentiment'].astype(int)

train = train.loc[:,['content', 'sentiment']]

train, _ = train_test_split(train, test_size=0.4, random_state=42, stratify=train['sentiment'])

In [12]:
train2 = pd.read_excel('/content/drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')

train2 = train2.loc[:,['Sentence', 'Emotion']]
train2 = train2.rename(columns={'Sentence': 'content', 'Emotion': 'sentiment'})
train2.loc[train2['sentiment'].isin(['분노', '슬픔', '혐오']), 'sentiment'] = -1
train2.loc[train2['sentiment']=='중립', 'sentiment'] = 0
train2.loc[train2['sentiment']=='행복', 'sentiment'] = 1

train2= train2.drop(train2[train2['sentiment'] == '공포'].index)
train2= train2.drop(train2[train2['sentiment'] == '놀람'].index)

In [16]:
train = pd.concat([train, train2], axis=0)
train = train.reset_index(drop=True)
train = train.drop(33420)  # 길이만 차지하는 쓸데없는 도배성 데이터 삭제 
train = train.drop(51875)
train = train.drop(57781)
train['sentiment'] = train['sentiment'].astype(int)
train['sentiment'] = train['sentiment'] + 1

In [19]:
train.sentiment.value_counts()  # 0 :부정 , 1: 중립 , 2 :긍정 

0    43661
2     9712
1     4830
Name: sentiment, dtype: int64

In [21]:
train_temp, valid_temp = train_test_split(train, test_size=0.1, random_state=42, stratify=train['sentiment'])

train_inputs = tokenizer(train_temp['content'].tolist(), padding=True, truncation=True, max_length = 512, return_tensors="pt")
valid_inputs = tokenizer(valid_temp['content'].tolist(), padding=True, truncation=True, max_length = 512, return_tensors="pt")

train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(train_temp['sentiment'].values))
valid_dataset = TensorDataset(valid_inputs['input_ids'], valid_inputs['attention_mask'], torch.tensor(valid_temp['sentiment'].values))


batch_size = 32

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
validation_dataloader = DataLoader(valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=batch_size)


save best validation model

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch


optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


lowest_val_loss = float('inf')
best_model = None

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    total_loss = 0

    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print("  Average training loss: {0:.2f}".format(avg_train_loss))


    if validation_dataloader is not None:
        model.eval()
        total_validation_loss = 0

        with torch.no_grad():
            for step, batch in enumerate(validation_dataloader):
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
                loss = outputs.loss
                total_validation_loss += loss.item()

        avg_validation_loss = total_validation_loss / len(validation_dataloader)
        print("  Average validation loss: {0:.2f}".format(avg_validation_loss))


        if avg_validation_loss < lowest_val_loss:
            print("Validation loss decreased ({:.2f} --> {:.2f}). Saving model ...".format(lowest_val_loss, avg_validation_loss))
            lowest_val_loss = avg_validation_loss
            # Save the model
            torch.save(model.state_dict(), '/content/best_model_state_neutral.bin')
            # If you want to save the entire model (not recommended due to space):
            # torch.save(model, 'best_model_full.bin')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


======== Epoch 1 / 4 ========


validation set 성능 확인 (안해도 됨)

In [ ]:
from sklearn.metrics import classification_report

# Prediction on test set
model.eval()

predictions, true_labels = [], []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    predictions.append(logits)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate metrics
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)

print(classification_report(flat_true_labels, flat_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9073
           1       0.94      0.94      0.94      1253

    accuracy                           0.99     10326
   macro avg       0.97      0.97      0.97     10326
weighted avg       0.99      0.99      0.99     10326



# 크롤링 데이터 에 적용

In [ ]:
df = pd.read_json('/content/drive/MyDrive/crawling_data.json')

df.loc[df.market == '자양시장', 'market'] = '자양전통시장'
df = df[df['content'] != '']
non_string_rows = df[df['content'].apply(lambda x: not isinstance(x, str))]

df.drop(non_string_rows.index, inplace=True)
df = df.reset_index(drop=True)

In [ ]:
# max length 512 ~ 200 바꿔가면서 해봐야됨 , train data의 tensor lengh가 180정도 됨 , test는 1600정도 되는거 200으로 truncate 진행 
test = tokenizer(df['content'].to_list(), padding=True, truncation=True, return_tensors="pt", max_length = 200)  

test.input_ids.shape

try full epoch model

In [ ]:
test_dataset = TensorDataset(test['input_ids'], test['attention_mask'])
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# Perform inference on test data
predictions = []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()

    predictions.append(logits)

# Flatten the predictions
flat_predictions = np.concatenate(predictions, axis=0)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
len(flat_predictions)

7919

In [ ]:
df['sentiment'] = flat_predictions

In [ ]:
df.loc[:,['content','sentiment']]

,content,sentiment
0,저는 2022년 5월 24일 13시 7분 자양전통시장 이용을 위해 자양전통시장 주차...,0
1,안녕하십니까 담당자님 자양전통시장을 자주 이용하는데 이번에 주차장을 이용하면서 불편...,0
2,청년 실업문제와 전통시장 부흥을 위해 화양시장 혹은 자양전통시장에 청년몰을 조성해주...,0
3,안녕하세요. 자양 1동 주민입니다. 최근 자양전통시장으로 진입하는 골목의 도로를 공...,0
4,첨부와 같이 2022년 06월 18일 망원시장 공영주차장의 시장할인 제도의 불합리한...,0
...,...,...
7914,화창한 봄기운 가득한 토요일요기요거 활짝 열었습니다봄이 성큼 다가왔어요오늘은 요기요...,1
7915,알록달록한 무지개 같은 닭강정을 파는 곳이다.망원시장에 방문하면 무조건 방문해야하기...,1
7916,나처럼 훈훈한 사람들만 방문해야 하는 곳이 있다.서울 마포구 망원역 망원시장 근처에...,1
7917,망원시장 같이 갈 친구 태그 !!#재슐랭망원 #망원시장 #망원시장투어근처 들를 일이...,1


try best model

In [ ]:
from transformers import BertForSequenceClassification
model_path = '/content/best_model_state_neutral.bin'


model = BertForSequenceClassification.from_pretrained(
    'skt/kobert-base-v1',
    num_labels=3, 
    output_attentions=False, 
    output_hidden_states=False  
)

model.load_state_dict(torch.load(model_path)) # val loss가 가장 낮았던 epoch의 model parameter들을 불러옴


print("Model loaded successfully!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
test_dataset = TensorDataset(test['input_ids'], test['attention_mask'])
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=32)

# Perform inference on test data
predictions = []

model.eval()
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()

    predictions.append(logits)

# Flatten the predictions
flat_predictions = np.concatenate(predictions, axis=0)

In [ ]:
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
df['sentiment'] = flat_predictions
df.loc[:,['content','sentiment']]

,content,sentiment
0,저는 2022년 5월 24일 13시 7분 자양전통시장 이용을 위해 자양전통시장 주차...,0
1,안녕하십니까 담당자님 자양전통시장을 자주 이용하는데 이번에 주차장을 이용하면서 불편...,0
2,청년 실업문제와 전통시장 부흥을 위해 화양시장 혹은 자양전통시장에 청년몰을 조성해주...,0
3,안녕하세요. 자양 1동 주민입니다. 최근 자양전통시장으로 진입하는 골목의 도로를 공...,0
4,첨부와 같이 2022년 06월 18일 망원시장 공영주차장의 시장할인 제도의 불합리한...,0
...,...,...
7914,화창한 봄기운 가득한 토요일요기요거 활짝 열었습니다봄이 성큼 다가왔어요오늘은 요기요...,1
7915,알록달록한 무지개 같은 닭강정을 파는 곳이다.망원시장에 방문하면 무조건 방문해야하기...,1
7916,나처럼 훈훈한 사람들만 방문해야 하는 곳이 있다.서울 마포구 망원역 망원시장 근처에...,0
7917,망원시장 같이 갈 친구 태그 !!#재슐랭망원 #망원시장 #망원시장투어근처 들를 일이...,1
